In [1]:
from utils import *
import os
import json

In [2]:
result_path = 'batch_5_src_no_sports/results'
result_files = [os.path.join(result_path, f"batch_{batch_id}_annotation.json") for batch_id in range(1,10)]
skip_sample_ids = {os.path.join(result_path, "batch_5_annotation.json"): list(range(40,50))}
selected_annotators = {
        # os.path.join(result_path, "batch_3_annotation.json"): ['yujia', 'rogger'],
        os.path.join(result_path, "batch_7_annotation.json"): ['yujia', 'manveer']
}

halu_eval = HaluEvaluator(result_files, skip_sample_ids=skip_sample_ids, selected_annotators=selected_annotators)
halu_eval.process_results()

In [3]:
print(json.dumps(halu_eval.compute_halu_rate(), indent=2))

number of records for openai/GPT-3.5-Turbo: 44
number of records for openai/gpt-4o: 44
number of records for Qwen/Qwen2.5-7B-Instruct: 44
number of records for microsoft/Phi-3-mini-4k-instruct: 44
number of records for cohere/command-r-08-2024: 44
number of records for meta-llama/Meta-Llama-3.1-8B-Instruct: 44
number of records for meta-llama/Meta-Llama-3.1-70B-Instruct: 44
number of records for google/gemini-1.5-flash-001: 44
number of records for Anthropic/claude-3-5-sonnet-20240620: 44
number of records for mistralai/Mistral-7B-Instruct-v0.3: 44
{
  "openai/gpt-4o": 38.64,
  "openai/GPT-3.5-Turbo": 50.0,
  "meta-llama/Meta-Llama-3.1-70B-Instruct": 52.27,
  "google/gemini-1.5-flash-001": 54.55,
  "meta-llama/Meta-Llama-3.1-8B-Instruct": 59.09,
  "Anthropic/claude-3-5-sonnet-20240620": 59.09,
  "Qwen/Qwen2.5-7B-Instruct": 65.91,
  "microsoft/Phi-3-mini-4k-instruct": 65.91,
  "cohere/command-r-08-2024": 77.27,
  "mistralai/Mistral-7B-Instruct-v0.3": 77.27
}
